<pre>
   _____                            _                __      ___     _                  _                        _ 
  / ____|                          | |               \ \    / (_)   (_)                | |                      | |
 | |     ___  _ __ ___  _ __  _   _| |_ ___ _ __      \ \  / / _ ___ _  ___  _ __      | |__   __ _ ___  ___  __| |
 | |    / _ \| '_ ` _ \| '_ \| | | | __/ _ \ '__|      \ \/ / | / __| |/ _ \| '_ \     | '_ \ / _` / __|/ _ \/ _` |
 | |___| (_) | | | | | | |_) | |_| | ||  __/ |          \  /  | \__ \ | (_) | | | |    | |_) | (_| \__ \  __/ (_| |
  \_____\___/|_| |_| |_| .__/ \__,_|\__\___|_|           \/   |_|___/_|\___/|_| |_|    |_.__/ \__,_|___/\___|\__,_|
  _______              | | __  __                                                    _                             
 |__   __|             |_||  \/  |                                                  | |                            
    | |_ __ ___  ___      | \  / | ___  __ _ ___ _   _ _ __ ___ _ __ ___   ___ _ __ | |_                           
    | | '__/ _ \/ _ \     | |\/| |/ _ \/ _` / __| | | | '__/ _ \ '_ ` _ \ / _ \ '_ \| __|                          
    | | | |  __/  __/     | |  | |  __/ (_| \__ \ |_| | | |  __/ | | | | |  __/ | | | |_                           
    |_|_|  \___|\___|     |_|  |_|\___|\__,_|___/\__,_|_|  \___|_| |_| |_|\___|_| |_|\__|                          
                                                                                                                   
                                                                                                                   

# Global Settings

In [10]:



## CONSTANTS

# Assets folder
ASSETS = "./assets"

# Video's
CALIBRATION_VIDEO = f"{ASSETS}/original/computervisie_2024/calibration.MP4"
EASTBOUND_VIDEO   = f"{ASSETS}/original/computervisie_2024/eastbound_20240319.MP4"
WESTBOUND_VIDEO   = f"{ASSETS}/original/computervisie_2024/westbound_20240319.MP4"

# Extracted frames
EXTRACTED_CALIBRATION = f"{ASSETS}/extracted_30/calibration/*.png"
EXTRACTED_WESTBOUND   = f"{ASSETS}/extracted_05/westbound_20240319/*.png"
EXTRACTED_EASTBOUND   = f"{ASSETS}/extracted_05/eastbound_20240319/*.png"

# Preliminary

### imports

In [3]:
# imports

import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import requests
from zipfile import ZipFile


OpenCV is not installed, installing now
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 16.0 MB/s eta 0:00:00:00:0100:01
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-8d5w_4p3
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-8d5w_4p3
  Resolved https://github.com/facebookresearch/detectron2.git to commit 0ae803b1449cd2d3f8fa1b7c0f59356db10b3083
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 16.1 MB/s eta 0:00:00
   ━━

In [6]:
# When running code in JupyterHub / Google Colab, Opencv might not be
# installed. This piece of code installs it when it is not yet available.
try:
    import cv2
    print("Succesfully imported OpenCV")
except ImportError:
    print("OpenCV is not installed, installing now")

    !pip install opencv-python

    import cv2

Succesfully imported OpenCV


In [ ]:
# installing detectron2
!pip install 'git+https://github.com/facebookresearch/detectron2.git'


from __future__ import  absolute_import

# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

import torch
import json
import gc

# import detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.video_visualizer import VideoVisualizer

In [6]:


url = "https://telin.ugent.be/nextcloud/index.php/s/rjgf4cw7m2iTGbx/download"

response = requests.get(url, stream=True)


# Check if the request was successful
if response.status_code == 200:
    # Open a file in binary write mode
    with open(f"{ASSETS}/videos.zip", "wb") as file:
        # Iterate over the response content by chunks
        for chunk in response.iter_content(chunk_size=1024):
            # Write the chunk to the file
            file.write(chunk)

    print("Zip file downloaded successfully.")

    with ZipFile(f"{ASSETS}/videos.zip", "r") as zip_ref:
        zip_ref.extractall(f"{ASSETS}/original")
else:
    print("Failed to download the zip file.")

Zip file downloaded successfully.


# Image Processing
## Frame Extraction

In this section, we extract individual frames from the video. This process involves loading the video file, iterating through each $k$ frames, and saving these frames as separate image files. Each extracted frame is named to include its frame number, ensuring a clear and organized sequence. Extracting frames allows for detailed analysis and processing of each moment captured in the video.

In [8]:
# frame extraction from video (keeping the original frame number)
# this could help us in the case where we cannot detect trees in certain frames, then we can use the original frame number to extract more frames from the original video in this time area

def extract_frames(video_path, capture_every_frame=5, output_folder=F"{ASSETS}/extracted"):
    # Adjust the output folder to include how many frames were skipped
    output_folder += f"_{capture_every_frame:02d}"

    # Skip if video has already been extracted
    if os.path.isdir(output_folder):
        print("skipping, video already extracted")
        return
    
    # Extract video name from path
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    
    # Create the directory if it doesn't exist
    os.makedirs(f"{output_folder}/{video_name}", exist_ok=True)

    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get total number of frames
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in {video_name}: {total_frames}")

    # Initialize the frame counter
    current_frame = 0

    # Process frames
    while current_frame < total_frames:
        video.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        success, image = video.read()

        # Check if the frame was successfully read
        # sometimes this fails, corrupt frames in vide? idk
        # using frames:05d (file_name_frame_00001.png) for easy sorting later on
        if success:
            # Save the frame with the actual frame number in the file name
            cv2.imwrite(f"{output_folder}/{video_name}/{video_name}_{current_frame:05d}.png", image)
            print(f"{output_folder}/{video_name}/{video_name}_{current_frame:05d}.png")
            # Skip to the next frame based on the specified interval
            current_frame += capture_every_frame
        else:
            # Output an error message if the frame failed to extract
            print(f"Frame {current_frame} failed to extract")
            # Try the next frame instead of skipping the interval because we couldn't read the current frame
            current_frame += 1


    # Release the video capture object
    video.release()


In [9]:
extract_frames(CALIBRATION_VIDEO, capture_every_frame=30)

skipping, video already extracted


In [13]:
extract_frames(EASTBOUND_VIDEO)

skipping, video already extracted


In [14]:
extract_frames(WESTBOUND_VIDEO)

skipping, video already extracted


## Calibration

In this section, we calculate the camera calibration matrices using Zhang's method, which is a widely used technique for camera calibration in computer vision. This method leverages multiple images of a known calibration pattern, in this case a chessboard, to estimate the camera's intrinsic and extrinsic parameters. By utilizing Zhang's method, this calibration process enables accurate determination of the camera's parameters, which are essential for correcting lens distortion and improving the accuracy of subsequent image processing tasks.

In [11]:
def calibrate_camera(calibration_images):
    objp = np.zeros((6*8, 3), np.float32)
    objp[:, :2] = np.mgrid[0:8, 0:6].T.reshape(-1, 2)

    objpoints = []
    imgpoints = []


    for img_path in calibration_images:
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        ret, corners = cv2.findChessboardCorners(gray, (8, 6), None)

        if ret:
            objpoints.append(objp)
            imgpoints.append(corners)


    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return ret, mtx, dist, rvecs, tvecs

In [12]:
calibration_images = glob(EXTRACTED_CALIBRATION)

ret, mtx, dist, rvecs, tvecs = calibrate_camera(calibration_images)

# save data for later use
os.makedirs(f"{ASSETS}/calibration_data", exist_ok=True)
np.save(f"{ASSETS}/calibration_data/intrinsic_parameters.npy", mtx)
np.save(f"{ASSETS}/calibration_data/distortion_coefficients.npy", dist)
np.save(f"{ASSETS}/calibration_data/rotation_vectors.npy", rvecs)
np.save(f"{ASSETS}/calibration_data/translation_vectors.npy", tvecs)

## Undistortion
After calibrating the camera and obtaining the necessary calibration matrices, the next step is to undistort the images. Lens distortion, which manifests as warping or curving of straight lines in images, is a common issue in photography, especially with wide-angle lenses. Using the calibration data derived from Zhang's method, we can correct this radial distortion and produce geometrically accurate images.

In [15]:
def undistort_images(images, output_loc=f"{ASSETS}/undistorted"):
    os.makedirs(f"{output_loc}", exist_ok=True)

    for img_path in images:
        img = cv2.imread(img_path)
        frame_name = os.path.splitext(os.path.basename(img_path))[0]
        
        undistorted_image = cv2.undistort(img, mtx, dist, None, mtx)
        cv2.imwrite(f"./{output_loc}/{frame_name}.png", undistorted_image)

In [20]:
westbound_images = glob(EXTRACTED_WESTBOUND)
eastbound_images = glob(EXTRACTED_EASTBOUND)

westbound_images.sort()
eastbound_images.sort()

undistort_images(westbound_images, output_loc="./assets/undistorted_05/westbound")
undistort_images(eastbound_images, output_loc="./assets/undistorted_05/eastbound")

# Tree Detection - TODO Clean Up

## PercepTree



In [ ]:
# UNUSED


# # Code from PercepTreeV1 use as test demo

# # local paths to model and image
# # model_name = 'X-101_RGB_60k.pth'
# # model_name = 'R-50_RGB_60k.pth'
# model_name = 'ResNext-101_fold_01.pth'
# image_path = './output/image_00000_RGB.png'

# if __name__ == "__main__":
#     torch.cuda.is_available()
#     logger = setup_logger(name=__name__)
    
#     # All configurables are listed in /repos/detectron2/detectron2/config/defaults.py        
#     cfg = get_cfg()
#     cfg.INPUT.MASK_FORMAT = "bitmask"
#     cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml"))
#     # cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"))
#     # cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
#     cfg.DATASETS.TRAIN = ()
#     cfg.DATASETS.TEST = ()
#     cfg.DATALOADER.NUM_WORKERS = 8
#     cfg.SOLVER.IMS_PER_BATCH = 8
#     cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # faster (default: 512)
#     cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (tree)
#     cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 1  
#     cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 5
#     cfg.MODEL.MASK_ON = True
    
#     cfg.OUTPUT_DIR = './output'
#     cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, model_name)
#     cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
#     # cfg.INPUT.MIN_SIZE_TEST = 0  # no resize at test time
    
#     # set detector
#     predictor_synth = DefaultPredictor(cfg)    
    
#     # set metadata
#     tree_metadata = MetadataCatalog.get("my_tree_dataset").set(thing_classes=["Tree"], keypoint_names=["kpCP", "kpL", "kpR", "AX1", "AX2"])
    
#     # inference
#     im = cv2.imread(image_path)
#     outputs_pred = predictor_synth(im)
#     v_synth = Visualizer(im[:, :, ::-1],
#                     metadata=tree_metadata, 
#                     scale=1,
#     )
#     out_synth = v_synth.draw_instance_predictions(outputs_pred["instances"].to("cpu"))

#     # Assuming out_synth.get_image() returns the image
#     image = out_synth.get_image()[:, :, ::-1]  # Assuming the image is in BGR format, converting it to RGB
    
#     plt.figure(figsize=(20, 10))
#     plt.imshow(image)
#     # plt.axis('off')  # Turn off axis
#     plt.show()

#     # Original code from demo, but this give errors in online notebooks, using above matplotlib instead
#     # cv2.imshow('predictions', out_synth.get_image()[:, :, ::-1])
#     # k = cv2.waitKey(0)
    
#     # cv2.destroyAllWindows()  
        

In [ ]:
# UNUSED

# predictions = outputs_pred["instances"].to("cpu")

In [ ]:
# UNUSED

# num_instances = len(predictions.pred_boxes)
# image_height = predictions.image_size[0]
# image_width = predictions.image_size[1]

# print(num_instances)
# print(image_height)
# print(image_width)

In [ ]:
# UNUSED

# predictions.get("pred_boxes").tensor.tolist()

In [ ]:
# UNUSED

# # Just showing the first two sets of 5 keypoints

# predictions.get("pred_keypoints").tolist()[:2]

In [ ]:
# Some global vars (temp location, will be moved into function call, def process_list_of_images(display_image, some_other_vars,...): )
display_image = False

# local paths to model and image
# model_name = 'X-101_RGB_60k.pth'
model_name = 'ResNext-101_fold_01.pth'
base_path = './assets/undistorted_05/eastbound'
image_dir_pattern = base_path + '/*.png'
output_dir = './assets/annotated_05/eastbound/'

# Ensure that the output directory exists, create it if necessary
os.makedirs(output_dir, exist_ok=True)

image_paths = glob(image_dir_pattern)
image_paths.sort()
print("Images to process:", len(image_paths))

# def process_list_of_images():
torch.cuda.is_available()
logger = setup_logger(name=__name__)

# All configurables are listed in /repos/detectron2/detectron2/config/defaults.py        
cfg = get_cfg()
cfg.INPUT.MASK_FORMAT = "bitmask"
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml"))
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"))
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ()
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 8
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # faster (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (tree)
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 1  
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 5
cfg.MODEL.MASK_ON = True

cfg.OUTPUT_DIR = './output'
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, model_name)
cfg.MODEL.WEIGHTS = f"./assets/models/{model_name}"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
# cfg.INPUT.MIN_SIZE_TEST = 0  # no resize at test time

# set detector
predictor_synth = DefaultPredictor(cfg)    

# set metadata
tree_metadata = MetadataCatalog.get("my_tree_dataset").set(thing_classes=["Tree"], keypoint_names=["kpCP", "kpL", "kpR", "AX1", "AX2"])

for image_path in image_paths:    
    file_name = image_path.split("/")[-1]
    output_path = output_dir + file_name
    # inference
    im = cv2.imread(image_path)
    outputs_pred = predictor_synth(im)
    v_synth = Visualizer(im[:, :, ::-1],
                    metadata=tree_metadata, 
                    scale=1,
    )
    predictions = outputs_pred["instances"].to("cpu")
    out_synth = v_synth.draw_instance_predictions(predictions)

    # Assuming out_synth.get_image() returns the image
    image = out_synth.get_image()[:, :, ::-1]  # Assuming the image is in BGR format, converting it to RGB
    
    # Save the image
    cv2.imwrite(output_path, image)
    print("Succesfully wrote image to:", output_path)
    
    # Convert the tensors to lists
    pred_boxes_list = predictions.get("pred_boxes").tensor.tolist()
    scores_list = predictions.get("scores").tolist()
    pred_keypoints_list = predictions.get("pred_keypoints").tolist()
    
    # .tolist() on a tensor is extremely slow, so saved it as npy instead (you can test this in a seperate cell and see how slow it is)
    # pred_masks_list = predictions.get("pred_masks_list").tolist()
    # pred_keypoints_heatmaps_list = predictions.get("pred_keypoint_heatmaps").tolist()
    pred_mask_numpy = predictions.get("pred_masks").numpy()
    pred_keypoint_heatmaps_numpy = predictions.get("pred_keypoint_heatmaps").numpy()

    # File base name
    file_base_name = file_name.split(".png")[0]
    # File names are kept in json as reference to numpy array file
    pred_masks_file_name = file_base_name + '_pred_mask.npy'
    pred_keypoint_heatmaps_file_name = file_base_name + '_pred_keypoints_heatmaps.npy'
    # Create full path that is used for saving the .npy files
    pred_mask_numpy_path = output_dir + pred_masks_file_name
    pred_keypoint_heatmaps_numpy_path = output_dir + pred_keypoint_heatmaps_file_name
    # Save the .npy arrays
    print("Saving:", pred_mask_numpy_path)
    np.save(pred_mask_numpy_path, pred_mask_numpy)
    print("Saving:", pred_keypoint_heatmaps_numpy_path)
    np.save(pred_keypoint_heatmaps_numpy_path, pred_keypoint_heatmaps_numpy)
    
    # Prepare a dictionary for JSON serialization
    json_data = {
        "pred_boxes": pred_boxes_list,
        "scores": scores_list,
        "pred_keypoints": pred_keypoints_list,
        "pred_masks": pred_masks_file_name,
        "pred_keypoint_heatmaps": pred_keypoint_heatmaps_file_name,
    }
    
    # Save to a JSON file    
    file_base_name = file_name.split(".png")[0]
    json_path = output_dir + file_base_name + '.json'
    with open(json_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)
    
    print(f"Data has been saved to {json_path}")    

    if display_image:        
        plt.figure(figsize=(20, 10))
        plt.imshow(out_synth.get_image())
        # plt.axis('off')  # Turn off axis
        plt.show()

    collected_garbage = gc.collect()
    print("Removed garbage:", collected_garbage)
    

Images to process: 3754
[05/19 12:12:35 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./assets/models/ResNext-101_fold_01.pth ...
Succesfully wrote image to: ./assets/annotated_05/eastbound/eastbound_20240319_00000.png
Saving: ./assets/annotated_05/eastbound/eastbound_20240319_00000_pred_mask.npy
Saving: ./assets/annotated_05/eastbound/eastbound_20240319_00000_pred_keypoints_heatmaps.npy
Data has been saved to ./assets/annotated_05/eastbound/eastbound_20240319_00000.json
Removed garbage: 8975
Succesfully wrote image to: ./assets/annotated_05/eastbound/eastbound_20240319_00005.png
Saving: ./assets/annotated_05/eastbound/eastbound_20240319_00005_pred_mask.npy
Saving: ./assets/annotated_05/eastbound/eastbound_20240319_00005_pred_keypoints_heatmaps.npy
Data has been saved to ./assets/annotated_05/eastbound/eastbound_20240319_00005.json
Removed garbage: 2691
Succesfully wrote image to: ./assets/annotated_05/eastbound/eastbound_20240319_00010.png
Saving: ./assets/

# Tree Triangulation (Colmap part goes here)

In [ ]:
# colmap



# Tree Mapping (notebook Robbe, Wout)

# Tree Measurement

In [6]:
# work of Thomas (depth images)